# IMAP Data Access API

The IMAP SDC provides a Data Access API to **query** and **download** data. There are three ways to use the API:

1. The `imap-data-access` command-line utility
2. The `imap_data_access` python package
3. Calls to the REST API directly

This notebook provies instructions and examples of how to use the command-line utility and python package. *For information on how to use the REST API directly, see https://imap-processing.readthedocs.io/en/latest/data-access/index.html*

## Data Directory

Before using the API, it is important to note that the folder structure for data files within the IMAP SDC is rigidly defined, so the data access will mimic that structure to make sure all data is stored in the same heirarchical structure as the SDC. This will enable seamless transition between a user's local system and the SDC.

The directory structure is as follows:

```
<IMAP_DATA_DIR>/
  imap/
    <instrument>/
      <data_level>/
        <year>/
          <month>/
            <filename>
```

for example:

```
/data/
  imap/
    swe/
      l0/
        2024/
          01/
            imap_swe_l0_sci_20240105_v001.pkts
```

By default, the `<IMAP_DATA_DIR>` is the user's current working directory. However, this can be changed by setting the `IMAP_DATA_DIR` environment variable:

In [1]:
!mkdir -p /Users/mabo8927/imap_data/
%env IMAP_DATA_DIR=/Users/mabo8927/imap_data
!echo $IMAP_DATA_DIR

env: IMAP_DATA_DIR=/Users/mabo8927/imap_data
/Users/mabo8927/imap_data


## Installation

The API can be installed via `pip`:

In [2]:
!pip install imap-data-access
!imap-data-access --version

  Using cached imap_data_access-0.13.1-py3-none-any.whl

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
imap-data-access 0.13.1


We leave it as an exercise to the user to install the package into their software environment of choice.

## Command Line Utility

The following section shows examples of how to use the API via the `imap-data-access` command line utility.

In [3]:
# Show help documentation
!imap-data-access -h

usage: imap-data-access [-h] [--version] [--api-key API_KEY]
                        [--data-dir DATA_DIR] [--url URL] [--debug] [-v]
                        {download,query,upload} ...

This command line program accesses the IMAP SDC APIs to query, download, and
upload data files.

positional arguments:
  {download,query,upload}
    download            Download a file from the IMAP SDC to the locally
                        configured data directory. Run 'download -h' for more
                        information.
    query               Query the IMAP SDC for files matching the query
                        parameters. The query parameters are optional, but at
                        least one must be provided. Run 'query -h' for more
                        information.
    upload              Upload a file to the IMAP SDC. This must be the full
                        path to the file. E.g. imap/mag/l0/2025/01/imap_mag_l0
                        _raw_20250101_v001.pkts. Run 'upload 

#### Querying

In [4]:
# Show query help documentation
!imap-data-access query -h

usage: imap-data-access query [-h]
                              [--instrument {codice,glows,hi,hit,idex,lo,mag,swapi,swe,ultra}]
                              [--data-level DATA_LEVEL]
                              [--descriptor DESCRIPTOR]
                              [--start-date START_DATE] [--end-date END_DATE]
                              [--repointing REPOINTING] [--version VERSION]
                              [--extension EXTENSION]
                              [--output-format {table,json}]
                              [--filename FILENAME]

Query the IMAP SDC for files matching the query parameters. The query
parameters are optional, but at least one must be provided.

optional arguments:
  -h, --help            show this help message and exit
  --instrument {codice,glows,hi,hit,idex,lo,mag,swapi,swe,ultra}
                        Name of the instrument
  --data-level DATA_LEVEL
                        Data level of the product (l0, l1a, l2, etc.)
  --descriptor DESCRI

In [5]:
# Query with a single parameter
!imap-data-access query --instrument swe

Found [23] matching files
|---------------------------------------------------------------------------------------------------------------|
| Instrument | Data Level | Descriptor | Start Date | Repointing | Version | Filename                           |
|---------------------------------------------------------------------------------------------------------------|
| swe        | l0         | raw        | 20240510   | None       | v001    | imap_swe_l0_raw_20240510_v001.pkts |
| swe        | l0         | raw        | 20240510   | None       | v002    | imap_swe_l0_raw_20240510_v002.pkts |
| swe        | l0         | raw        | 20240510   | None       | v007    | imap_swe_l0_raw_20240510_v007.pkts |
| swe        | l0         | raw        | 20240510   | None       | v008    | imap_swe_l0_raw_20240510_v008.pkts |
| swe        | l0         | raw        | 20240510   | None       | v009    | imap_swe_l0_raw_20240510_v009.pkts |
| swe        | l0         | raw        | 20240510   | None    

In [6]:
# Query with multiple parameters
!imap-data-access query --instrument swe --data-level l1b --start-date 20240510 --end-date 20240510

Found [4] matching files
|---------------------------------------------------------------------------------------------------------------|
| Instrument | Data Level | Descriptor | Start Date | Repointing | Version | Filename                           |
|---------------------------------------------------------------------------------------------------------------|
| swe        | l1b        | sci        | 20240510   | None       | v001    | imap_swe_l1b_sci_20240510_v001.cdf |
| swe        | l1b        | sci        | 20240510   | None       | v002    | imap_swe_l1b_sci_20240510_v002.cdf |
| swe        | l1b        | sci        | 20240510   | None       | v003    | imap_swe_l1b_sci_20240510_v003.cdf |
| swe        | l1b        | sci        | 20240510   | None       | v012    | imap_swe_l1b_sci_20240510_v012.cdf |
|---------------------------------------------------------------------------------------------------------------|


In [7]:
# Change the output format
!imap-data-access query --instrument swe --data-level l1b --start-date 20240510 --end-date 20240510 --output-format json

[{'file_path': 'imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v001.cdf', 'instrument': 'swe', 'data_level': 'l1b', 'descriptor': 'sci', 'start_date': '20240510', 'repointing': None, 'version': 'v001', 'extension': 'cdf', 'ingestion_date': '2024-10-07 17:06:50'}, {'file_path': 'imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v002.cdf', 'instrument': 'swe', 'data_level': 'l1b', 'descriptor': 'sci', 'start_date': '20240510', 'repointing': None, 'version': 'v002', 'extension': 'cdf', 'ingestion_date': '2024-10-07 20:07:44'}, {'file_path': 'imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v003.cdf', 'instrument': 'swe', 'data_level': 'l1b', 'descriptor': 'sci', 'start_date': '20240510', 'repointing': None, 'version': 'v003', 'extension': 'cdf', 'ingestion_date': '2024-11-08 18:26:10'}, {'file_path': 'imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v012.cdf', 'instrument': 'swe', 'data_level': 'l1b', 'descriptor': 'sci', 'start_date': '20240510', 'repointing': None, 'version': 'v012', 'extensio

In [8]:
# Query based on a filename
!imap-data-access query --filename imap_swe_l1b_sci_20240510_v012.cdf

Found [1] matching files
|---------------------------------------------------------------------------------------------------------------|
| Instrument | Data Level | Descriptor | Start Date | Repointing | Version | Filename                           |
|---------------------------------------------------------------------------------------------------------------|
| swe        | l1b        | sci        | 20240510   | None       | v012    | imap_swe_l1b_sci_20240510_v012.cdf |
|---------------------------------------------------------------------------------------------------------------|


#### Downloading

In [9]:
# Show download help documentation
!imap-data-access download -h

usage: imap-data-access download [-h] file_path

Download a file from the IMAP SDC to the locally configured data directory.

positional arguments:
  file_path   This must be the full path to the file. E.g.
              imap/mag/l0/2025/01/imap_mag_l0_raw_20250101_v001.pkts

optional arguments:
  -h, --help  show this help message and exit


##### Science Files

In [10]:
# Download a science file
!imap-data-access download imap_swe_l1b_sci_20240510_v012.cdf

# Check to see that the file exists
!ls -lt $IMAP_DATA_DIR/imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v012.cdf

Successfully downloaded the file to: /Users/mabo8927/imap_data/imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v012.cdf
-rw-r--r--  1 mabo8927  2260  304761 Feb 18 15:12 /Users/mabo8927/imap_data/imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v012.cdf


##### Ancillary Files

In [11]:
# Download an ancillary file
!imap-data-access download imap_mag_calibration_20240229_v001.cdf

# Check to see that the file exists
!ls -lt $IMAP_DATA_DIR/imap/ancillary/mag/imap_mag_calibration_20240229_v001.cdf

Successfully downloaded the file to: /Users/mabo8927/imap_data/imap/ancillary/mag/imap_mag_calibration_20240229_v001.cdf
-rw-r--r--  1 mabo8927  2260  3269 Feb 18 15:12 /Users/mabo8927/imap_data/imap/ancillary/mag/imap_mag_calibration_20240229_v001.cdf


## Python Package
The following section shows examples of how to use the API via the `imap_data_access` Python package.

In [12]:
# import the package
import imap_data_access

The `<IMAP_DATA_DIR>` can be set via the `config` dictionary:

In [13]:
from pathlib import Path

imap_data_access.config["DATA_DIR"] = Path.home() / "imap_data"

#### Querying

In [14]:
results = imap_data_access.query(instrument="swe", data_level="l0")
for result in results:
    print(result)

{'file_path': 'imap/swe/l0/2024/05/imap_swe_l0_raw_20240510_v001.pkts', 'instrument': 'swe', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20240510', 'repointing': None, 'version': 'v001', 'extension': 'pkts', 'ingestion_date': '2024-10-07 17:05:14'}
{'file_path': 'imap/swe/l0/2024/05/imap_swe_l0_raw_20240510_v002.pkts', 'instrument': 'swe', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20240510', 'repointing': None, 'version': 'v002', 'extension': 'pkts', 'ingestion_date': '2024-10-07 20:06:01'}
{'file_path': 'imap/swe/l0/2024/05/imap_swe_l0_raw_20240510_v007.pkts', 'instrument': 'swe', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20240510', 'repointing': None, 'version': 'v007', 'extension': 'pkts', 'ingestion_date': '2024-12-06 18:31:33'}
{'file_path': 'imap/swe/l0/2024/05/imap_swe_l0_raw_20240510_v008.pkts', 'instrument': 'swe', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20240510', 'repointing': None, 'version': 'v008', 'extension': 'pkts

#### Download a file

In [15]:
file_path = imap_data_access.download("imap_swe_l0_raw_20241212_v000.pkts")
print(file_path)

/Users/mabo8927/imap_data/imap/swe/l0/2024/12/imap_swe_l0_raw_20241212_v000.pkts


#### Bonus: validate/construct a file path

In [16]:
science_file = imap_data_access.ScienceFilePath("imap_mag_l0_raw_20250502_v001.pkts")
print(science_file.start_date)
print(science_file.descriptor)

filepath = science_file.construct_path()
print(filepath)

20250502
raw
/Users/mabo8927/imap_data/imap/mag/l0/2025/05/imap_mag_l0_raw_20250502_v001.pkts


## Future work

The SDC continues to develop the API for the greater needs of the IMAP community. We currently have plans to add the following features (subject to change):

- Add support for querying ancillary files
- Add support for querying and downloading SPICE files
- Add query parameter for querying based on repointing
- Create a production instance of the API

## Ideas/requests for other API features or endpoints?

We would like to gather user feedback on how we could improve or expand upon the existing API. What API endpoints could we add? What kind of information would be useful to get from the API?

For feature requests and bug reports, please open a [new issue](https://github.com/IMAP-Science-Operations-Center/imap-data-access/issues/new/choose) in the Data Access API repository.
